# 시간대별 혼잡도와 직장인구 상관분석

이 노트북에서는 지하철 역별 시간대별 혼잡도와 해당 역이 위치한 행정동의 직장인구 데이터를 사용하여 상관관계를 분석합니다.

## 분석 목표
- **상업 지구(직장인구 밀집 지역)**가 지하철 혼잡도에 미치는 영향을 파악합니다.
- 출근 시간대(07~09시)와 퇴근 시간대(17~19시)의 패턴 차이를 확인합니다.

In [ ]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# 한글 폰트 설정 (필요시)
# import matplotlib.pyplot as plt
# plt.rc('font', family='Malgun Gothic')

DB_PATH = '../db/subway.db'

## 1. 데이터 로드

In [ ]:
def load_data():
    conn = sqlite3.connect(DB_PATH)
    
    try:
        # 1. 혼잡도 데이터 로드
        # 2024년 데이터만 사용 (최신)
        query_congestion = """
        SELECT 
            station_code, 
            day_of_week, 
            time_slot, 
            congestion_level
        FROM Station_Congestion
        WHERE day_of_week = 0 -- 평일만 분석
        """
        df_congestion = pd.read_sql(query_congestion, conn)
        
        # 2. 직장인구 데이터 로드 (가장 최근 분기)
        # quarter_code가 가장 큰(최신) 데이터만 가져옵니다.
        query_workplace = """
        SELECT 
            admin_dong_code, 
            admin_dong_name, 
            total_pop as workplace_pop
        FROM Dong_Workplace_Population
        WHERE quarter_code = (SELECT MAX(quarter_code) FROM Dong_Workplace_Population)
        """
        df_workplace = pd.read_sql(query_workplace, conn)
        
        # 3. 역-행정동 매핑 정보 로드
        query_mapping = """
        SELECT 
            station_code, 
            station_id,
            admin_dong_code,
            admin_dong_name
        FROM Station_Routes
        WHERE admin_dong_code IS NOT NULL
        """
        df_mapping = pd.read_sql(query_mapping, conn)
        
        # 4. 역 정보 (이름)
        query_stations = "SELECT station_id, station_name_kr FROM Stations"
        df_stations = pd.read_sql(query_stations, conn)
        
        # 매핑 정보에 역 이름 추가
        df_mapping = df_mapping.merge(df_stations, on='station_id', how='left')
        
    finally:
        conn.close()
        
    return df_congestion, df_workplace, df_mapping

df_congestion, df_workplace, df_mapping = load_data()

print(f"Congestion Data: {df_congestion.shape}")
print(f"Workplace Data: {df_workplace.shape}")
print(f"Mapping Data: {df_mapping.shape}")

display(df_congestion.head())
display(df_workplace.head())

## 2. 데이터 전처리 및 병합

- 혼잡도 데이터에 행정동 정보를 매핑합니다.
- 역이 여러 행정동에 걸쳐있는 경우는 드물지만, `Station_Routes` 기준으로 1:1 매핑을 가정하거나 중복 시 첫 번째를 사용합니다.
- 행정동별로 평균 혼잡도를 집계합니다.

In [ ]:
# [FIX] 행정동 코드 자릿수 매칭 (10자리 -> 8자리)
# Station_Routes는 10자리, Dong_Workplace_Population은 8자리를 사용하므로 앞 8자리만 사용하여 매칭
df_mapping['admin_dong_code'] = df_mapping['admin_dong_code'].astype(str).str[:8]
df_workplace['admin_dong_code'] = df_workplace['admin_dong_code'].astype(str).str.strip()

# 혼잡도 데이터에 매핑 정보 결합
df_merged = df_congestion.merge(df_mapping[['station_code', 'admin_dong_code', 'station_name_kr']], on='station_code', how='inner')

# 행정동별, 시간대별 평균 혼잡도 계산
df_dong_congestion = df_merged.groupby(['admin_dong_code', 'time_slot'])['congestion_level'].mean().reset_index()

# 직장인구 데이터 결합
df_final = df_dong_congestion.merge(df_workplace, on='admin_dong_code', how='inner')

# time_slot을 보기 좋게 변환 (예: 1 -> 05:30, 2 -> 06:00 ...)
# 가정: time_slot 1은 05:30부터 30분 단위라고 가정 (데이터 확인 필요)
# 일반적인 서울교통공사 데이터 기준: 1=05:30, 2=06:00, ...
def convert_time_slot(slot):
    start_hour = 5
    start_min = 30
    
    total_min = start_min + (slot - 1) * 30
    hour = start_hour + total_min // 60
    minute = total_min % 60
    
    return f"{int(hour):02d}:{int(minute):02d}"

df_final['time_label'] = df_final['time_slot'].apply(convert_time_slot)

print(f"Final Dataset Shape: {df_final.shape}")
display(df_final.head())

## 3. 상관관계 분석

시간대별로 **혼잡도**와 **직장인구수**의 상관계수를 계산합니다.

In [ ]:
# 시간대별 상관계수 계산
correlations = []

for time_slot in sorted(df_final['time_slot'].unique()):
    subset = df_final[df_final['time_slot'] == time_slot]
    if len(subset) > 10:
        corr = subset['congestion_level'].corr(subset['workplace_pop'])
        time_label = subset['time_label'].iloc[0]
        correlations.append({'time_slot': time_slot, 'time_label': time_label, 'correlation': corr})
    
df_corr = pd.DataFrame(correlations)

display(df_corr)

## 4. 시각화

In [ ]:
# 4.1 시간대별 혼잡도와 직장인구 상관계수 변화 (Line Plot)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_corr['time_label'],
    y=df_corr['correlation'],
    mode='lines+markers',
    name='상관계수',
    line=dict(color='royalblue', width=3),
    marker=dict(size=8)
))

fig.update_layout(
    title='시간대별 혼잡도와 직장인구수 상관관계 변화',
    xaxis_title='시간대',
    yaxis_title='상관계수 (Pearson)',
    template='plotly_white',
    xaxis_tickangle=-45
)

fig.show()


In [ ]:
# 4.2 출근 시간대(08:00~09:00) vs 퇴근 시간대(18:00~19:00) 산점도 비교

# 출근 피크: 08:00 (slot 계산 필요)
# 1: 05:30, 2: 06:00, 3: 06:30, 4: 07:00, 5: 07:30, 6: 08:00 ... 
# 공식: slot = (hour - 5) * 2 - (1 if min==0 else 0) ... 복잡하니 위 함수 역산하거나 직접 찾음
# 08:00 -> 5:30(1), 6:00(2), 6:30(3), 7:00(4), 7:30(5), 8:00(6)
# 18:00 -> ... 18:00 해당 슬롯 찾기

target_times = ['08:00', '18:00']
sub_df = df_final[df_final['time_label'].isin(target_times)]

fig = px.scatter(
    sub_df,
    x='workplace_pop',
    y='congestion_level',
    color='time_label',
    hover_data=['admin_dong_name'],
    title='출퇴근 시간대 직장인구수 vs 혼잡도 분포',
    labels={'workplace_pop': '직장인구수', 'congestion_level': '혼잡도', 'time_label': '시간대'},
    trendline='ols' # 추세선 추가
)

fig.update_layout(template='plotly_white')
fig.show()

## 5. 결론 및 인사이트

- **출근 시간대**: 직장인구가 많은 지역일수록 도착하는 인원이 많아 하차 혼잡도가 높을 것으로 예상되지만, 혼잡도 데이터가 '차내 혼잡도'라면 해당 역을 지나가는 인원도 포함되므로 해석에 주의가 필요합니다.
- **퇴근 시간대**: 직장인구가 많은 지역에서 승차하는 인원이 많아 혼잡도가 높아지는 경향이 있는지 확인합니다.

위 그래프를 통해 시간대별 상관성의 흐름을 파악할 수 있습니다.